In [1]:
import numpy as np
import scipy.stats as stats

In [2]:
import xgboost as xgb

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [5]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [6]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [7]:
processed = pd.read_csv('features_03.csv')

In [8]:
processed = processed.set_index('person')
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,visitas_por_dia,vio_celulares,region,is_new
person,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral,2.000000,False,Unknown,True
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct,1.214286,True,Rio Grande do Sul,True
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid,1.000000,True,Minas Gerais,True
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct,0.600000,True,Sao Paulo,True
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid,1.000000,True,Minas Gerais,True


In [9]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [10]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

/home/javier/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,visitas_por_dia,vio_celulares,region,is_new
person,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.000000,Referral,2.000000,False,Unknown,True
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.000000,Paid,1.000000,True,Minas Gerais,True
001802e4,ad campaign hit,iPhone 6S,Smartphone,0,1,4,19,1,0,19.000000,Paid,1.000000,True,Rio de Janeiro,True
0019e639,viewed product,Motorola Moto G2 3G Dual,Computer,2,19,189,471,115,9,4.095652,Paid,0.165217,True,Sao Paulo,True
001b0bf9,checkout,Samsung Galaxy S8 Plus,Ninguno,1,0,2,7,1,9,7.000000,Ninguno,0.000000,True,Unknown,False


## Mean encoding

In [12]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [13]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.007335,0.042654,0.069914,0,2,0,6,1,14,6.000000,0.065251,2.000000,False,0.046687,True
1,0.053592,0.049793,0.034791,0,1,3,17,1,2,17.000000,0.042291,1.000000,True,0.058204,True
2,0.018508,0.034037,0.034791,0,1,4,19,1,0,19.000000,0.042291,1.000000,True,0.049149,True
3,0.055779,0.026954,0.069914,2,19,189,471,115,9,4.095652,0.042291,0.165217,True,0.053818,True
4,0.007335,0.054054,0.017361,1,0,2,7,1,9,7.000000,0.017361,0.000000,True,0.046687,False


In [57]:
params={
    'n_estimators':stats.randint(150,500),
    'learning_rate':stats.uniform(0.01,0.3),
    'subsample':stats.uniform(0.3,0.7),
    'min_child_weight':[1,5,10],
    'max_depth':[3,10,6],
    'gamma':[0,1,5],
    'colsample_bytree':stats.uniform(0.4,0.6)
}
grid = RandomizedSearchCV(xgb.XGBClassifier(n_jobs=-1),
                          param_distributions=params,
                          scoring='roc_auc',
                          cv=2,
                          verbose=1,
                          n_iter=25)

In [58]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [ ]:
grid.fit(xtr,ytr)

In [60]:
grid.score(X=xte,y=yte)

0.7843261401449648

In [61]:
grid.best_params_

{'colsample_bytree': 0.5260820206459875,
 'gamma': 5,
 'learning_rate': 0.03944207589130514,
 'max_depth': 10,
 'min_child_weight': 10,
 'n_estimators': 172,
 'subsample': 0.9527467047072764}

## Prediccion

In [62]:
predictor = grid.best_estimator_

In [63]:
predictor.feature_importances_

array([0.06796116, 0.13713592, 0.05946602, 0.04429612, 0.03519417,
       0.0618932 , 0.09223301, 0.07099514, 0.13167475, 0.0934466 ,
       0.03762136, 0.08798543, 0.        , 0.08009709, 0.        ],
      dtype=float32)

In [64]:
predictor.fit(encoded_training,training_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5260820206459875, gamma=5,
       learning_rate=0.03944207589130514, max_delta_step=0, max_depth=10,
       min_child_weight=10, missing=None, n_estimators=172, n_jobs=-1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.9527467047072764)

In [65]:
prediction_submit = predictor.predict_proba(encoded_predict)[:,1]

In [66]:
predict_features['label'] = prediction_submit

In [67]:
subm = predict.merge(predict_features['label'].reset_index(),how='left',on='person')

In [68]:
subm = subm.set_index('person')

In [69]:
subm.to_csv('submit.csv')